# Data sources, data gathering and edition

Here is the list of the websites that we gathered our data from. The wikipedia website of the arrondissements (arrondissement is the french equivalent to neighborhood or borough) and two open data websites from the Paris city hall

https://opendata.paris.fr/explore/dataset/arrondissements/api/?disjunctive.c_ar&disjunctive.c_arinsee&disjunctive.l_ar&basemap=jawg.dark&location=12,48.85889,2.34692&dataChart=eyJxdWVyaWVzIjpbeyJjb25maWciOnsiZGF0YXNldCI6ImFycm9uZGlzc2VtZW50cyIsIm9wdGlvbnMiOnsiZGlzanVuY3RpdmUuY19hciI6dHJ1ZSwiZGlzanVuY3RpdmUuY19hcmluc2VlIjp0cnVlLCJkaXNqdW5jdGl2ZS5sX2FyIjp0cnVlfX0sImNoYXJ0cyI6W3siYWxpZ25Nb250aCI6dHJ1ZSwidHlwZSI6ImNvbHVtbiIsImZ1bmMiOiJBVkciLCJ5QXhpcyI6Im5fc3FfYXIiLCJzY2llbnRpZmljRGlzcGxheSI6dHJ1ZSwiY29sb3IiOiJyYW5nZS1jdXN0b20ifV0sInhBeGlzIjoibF9hciIsIm1heHBvaW50cyI6NTAsInNvcnQiOiIiLCJzZXJpZXNCcmVha2Rvd24iOiJzdXJmYWNlIn1dLCJ0aW1lc2NhbGUiOiIiLCJkaXNwbGF5TGVnZW5kIjp0cnVlLCJhbGlnbk1vbnRoIjp0cnVlfQ%3D%3D
https://fr.wikipedia.org/wiki/Arrondissements_de_Paris
https://www.data.gouv.fr/fr/datasets/arrondissements-1/

First of all, we import the all the modules required for our study

In [1]:
import numpy as np 
import pandas as pd
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import sklearn.neighbors
import folium
from bs4 import BeautifulSoup
import geocoder
import shapely
from shapely.geometry import Point, Polygon
import geopandas as gpd
from geopandas import GeoSeries
from fiona.crs import from_epsg
import pysal.viz.mapclassify as mc
import config
import json

print ('Imported!')

Imported!


C:\Users\Utilisateur\anaconda3\envs\geopandas_env\lib\site-packages\pysal\explore\segregation\network\network.py:15: UserWarning: You need pandana and urbanaccess to work with segregation's network module
You can install them with  `pip install urbanaccess pandana` or `conda install -c udst pandana urbanaccess`
  warn(
C:\Users\Utilisateur\anaconda3\envs\geopandas_env\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


We gather the information that we need from the data sources. First, we scrap the administrative data from the wikipedia site about the arrondissements in Paris and then store it into a pandas dataframe. Among other information, we collect the population density data for each arrondissement. We'd like to study the possibility of building a park in a neighborhood with a high population density rather than in one with a low value.

In [2]:
# Response in the form of html
wikiurl="https://fr.wikipedia.org/wiki/Arrondissements_de_Paris"
table_class="wikitable sortable jquery-tablesorter"
response=requests.get(wikiurl)

# Parse data from the html table which contains the data into a beautifulsoup object
soup = BeautifulSoup(response.text, 'html.parser')
paris_data=soup.find_all("table")[3]

# Converting data into a pandas dataframe
admi=pd.read_html(str(paris_data))
admi=pd.DataFrame(admi[0])

    # Data cleaning and preparation
admi= admi.drop([admi.index[20], admi.index[21], admi.index[22]]) # Dropping the last three rows

    # Choosing the relevant information, data from 2015
cols = ['Nom','Pop_2015', 'Dens_2015']   
admi_sel = pd.DataFrame(columns = cols)
admi_sel['Pop_2015']=admi['Population (municipale pour 2010 et 2015)']['2015']
admi_sel['Dens_2015']=admi['Densité (hab./km²)']['2015']
admi_sel['Nom']=admi['Nom']['Nom']
admi_sel['num']=range(1, 1+len(admi_sel)) # Add a consecutive series of numbers from 1 to 20 which corresponds to the arrondissement number

# Converting data from numeric columns into integers manually. The blank space between numbers from wikipedia makes the data not usable
# There is a lot of documentation about converting obj to int or float, adding characters in strings, etc, but I couldn't find any reference
# that handled THAT blank space
admi_sel['Pop_2015'] = [16545,20796,35049,27146,59333,42428,54133,36694,59408,91770,149834,142340,183216,139992,234994,165487,168533,197580,185654,195556]
admi_sel['Dens_2015'] = [9041,21006,29956,16966,23359,19734,13235,9457,27251,31754,40827,22345,25625,24821,27712,20921,29724,32875,27342,32702]

admi_sel.sort_values(by='Dens_2015', ascending=False)

,Nom,Pop_2015,Dens_2015,num
10,Popincourt,149834,40827,11
17,Buttes-Montmartre,197580,32875,18
19,Ménilmontant,195556,32702,20
9,"Entrepôt, anciennement Enclos Saint-Laurent",91770,31754,10
2,Temple,35049,29956,3
16,Batignolles-Monceau,168533,29724,17
14,Vaugirard,234994,27712,15
18,Buttes-Chaumont,185654,27342,19
8,Opéra[note 1],59408,27251,9
12,Gobelins,183216,25625,13


We get the json file from the Paris city hall open data website. Then we create a dataframe that contains the geographic center for each arrondissement

In [3]:
# Importing data from the the Paris open data API and importing them to a dataframe
resp = requests.get("https://opendata.paris.fr/api/records/1.0/search/?dataset=arrondissements&q=&rows=20&facet=c_ar&facet=c_arinsee&facet=l_ar")

data = resp.json()
raw_data = pd.json_normalize(data['records'])
raw_data = raw_data[["fields.l_ar","fields.geom_x_y","fields.geom.coordinates", "fields.l_aroff",
                  "fields.c_ar","geometry.coordinates"]]

# Editing dataframe columns
edit_data = raw_data.rename(columns={'fields.l_ar':'cod_ar', 'fields.geom_x_y':'center', 'fields.geom.coordinates':'poly',
                               'fields.l_aroff':'name', 'fields.c_ar':'num', 'geometry.coordinates':'geometry'})
edit_data.head()

,cod_ar,center,poly,name,num,geometry
0,1er Ardt,"[48.8625627018, 2.33644336205]","[[[2.328007329038849, 48.86991742140716], [2.3...",Louvre,1,"[2.33644336205, 48.8625627018]"
1,2ème Ardt,"[48.8682792225, 2.34280254689]","[[[2.351518483670822, 48.8644258050741], [2.35...",Bourse,2,"[2.34280254689, 48.8682792225]"
2,19ème Ardt,"[48.8870759966, 2.38482096015]","[[[2.38942869613216, 48.901218118234496], [2.3...",Buttes-Chaumont,19,"[2.38482096015, 48.8870759966]"
3,11ème Ardt,"[48.8590592213, 2.3800583082]","[[[2.3962365763098292, 48.85415458748718], [2....",Popincourt,11,"[2.3800583082, 48.8590592213]"
4,10ème Ardt,"[48.8761300365, 2.36072848785]","[[[2.364685889119478, 48.88436904340793], [2.3...",Entrepôt,10,"[2.36072848785, 48.8761300365]"


In [5]:
# Creation of a dataframe that contains the geoinformation for the centers of each arrondissement

# Dropping columns that are not needed
coor_c = edit_data.drop(['poly'], axis=1)

# Creation of 2 new fields, lat and lon, XY point coordinates
coor_c['lat'] = ''
coor_c['lon'] = ''

#Copying coordinates of the centers into the new columns
j = 0
for i in coor_c['center']: # Iteration over the columns containing the data
    latlon = i
    lat = i[0]
    coor_c.loc[j, 'lat'] = lat 
    lon = i[1]
    coor_c.loc[j, 'lon'] = lon
    j = j+1
coor_c.head()

,cod_ar,center,name,num,geometry,lat,lon
0,1er Ardt,"[48.8625627018, 2.33644336205]",Louvre,1,"[2.33644336205, 48.8625627018]",48.862563,2.336443
1,2ème Ardt,"[48.8682792225, 2.34280254689]",Bourse,2,"[2.34280254689, 48.8682792225]",48.868279,2.342803
2,19ème Ardt,"[48.8870759966, 2.38482096015]",Buttes-Chaumont,19,"[2.38482096015, 48.8870759966]",48.887076,2.384821
3,11ème Ardt,"[48.8590592213, 2.3800583082]",Popincourt,11,"[2.3800583082, 48.8590592213]",48.859059,2.380058
4,10ème Ardt,"[48.8761300365, 2.36072848785]",Entrepôt,10,"[2.36072848785, 48.8761300365]",48.87613,2.360728


From a shapefile, we extract the contour lines of the polygons for each arrondissement/neighborhood and store it into a dataframe. Then we merge it to a single dataframe contaning the administrative information (density of population, name, population, etc.) and the geographic information.

In [6]:
# Creating a geodataframe (coor_p) from the arrondissements.shp. It is easier to work with shp files than json when handling polygons
arr_fp = "C:/Users/Utilisateur/Documents/Formacion/IBM_datascience/Capstone_project/Final_project/arrondissements.shp"
coor_p= gpd.read_file(arr_fp)
coor_p['geometry'] = coor_p['geometry'].to_crs(epsg=4326) #Defining the coordinate refence system (CRS)
coor_p['geoid'] = coor_p.index.astype(str)
coor_p = coor_p[['geoid','c_ar','geometry']]
coor_p_jsontxt = coor_p.to_json()

#Editing and dropping columns
coor_p['num'] = coor_p['c_ar'].astype(int)
coor_p = coor_p.drop(['c_ar'], axis =1)

# Merging the two dataframes for graphic representation (on population density 2015)
coor_p=pd.merge(coor_p, admi_sel, on ='num', how='left')

coor_p.head()

,geoid,geometry,num,Nom,Pop_2015,Dens_2015
0,0,"POLYGON ((2.36383 48.86750, 2.36389 48.86747, ...",3,Temple,35049,29956
1,1,"POLYGON ((2.32090 48.86306, 2.32094 48.86305, ...",7,Palais-Bourbon,54133,13235
2,2,"POLYGON ((2.36443 48.84614, 2.36484 48.84584, ...",5,Panthéon,59333,23359
3,3,"POLYGON ((2.32584 48.86956, 2.32569 48.86954, ...",8,Élysée,36694,9457
4,4,"POLYGON ((2.41388 48.83357, 2.41401 48.83357, ...",12,Reuilly (hors bois de Vincennes),142340,22345


In [12]:
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Querying results from Foursquare API from all neighbourhoods
Set up the search limit to 1500m from the center of the polygon
The function is run twice, once for the gardens and once for the metro
stations. After multiple test, we saw that querying gave better results
and more accurate than using the API category code. As it isn't possible
to do multiple queries, we're forced to run the function twice.

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Creation of the API request URL to look for gardens
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=Garden'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
garden_venues = getNearbyVenues(names=coor_c['name'],
                                   latitudes=coor_c['lat'],
                                   longitudes=coor_c['lon']
                                  )

Louvre
Bourse
Buttes-Chaumont
Popincourt
Entrepôt
Batignolles-Monceau
Ménilmontant
Vaugirard
Reuilly
Hôtel-de-Ville
Observatoire
Gobelins
Luxembourg
Panthéon
Élysée
Passy
Opéra
Buttes-Montmartre
Temple
Palais-Bourbon


In [15]:
#print(garden_venues['Venue Category'].unique())
#print(garden_venues.shape)

#Data cleaning: not gardens or parks
garden_paris = garden_venues.loc[(garden_venues['Venue Category']=='Garden') | (garden_venues['Venue Category']=='Park')] 
garden_paris = garden_paris.rename(columns ={'Venue': 'garden','Venue Latitude': 'lat_g', 'Venue Longitude': 'lon_g'})
garden_paris = garden_paris[~garden_paris.garden.str.contains('3C - Conception- Coordination- Construction')] # These two locations are 
garden_paris = garden_paris[~garden_paris.garden.str.contains('Eymin paysagistes')] # classified as "Garden". They have to be removed manually

#print(garden_paris.shape)
garden_paris.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,garden,lat_g,lon_g,Venue Category
0,Louvre,48.862563,2.336443,Jardin du Palais Royal,48.864941,2.337728,Garden
1,Louvre,48.862563,2.336443,Jardin des Tuileries,48.863642,2.326484,Garden
2,Louvre,48.862563,2.336443,Jardins du Carrousel (Jardin du Carrousel),48.862056,2.332445,Garden
5,Louvre,48.862563,2.336443,Place Dauphine,48.856640,2.342362,Garden
6,Louvre,48.862563,2.336443,Square de la Tour Saint-Jacques,48.857882,2.348757,Garden


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # Creation of the API request URL to look for transports
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=Metro'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
transport_venues = getNearbyVenues(names=coor_c['name'],
                                   latitudes=coor_c['lat'],
                                   longitudes=coor_c['lon']
                                  )

Louvre
Bourse
Buttes-Chaumont
Popincourt
Entrepôt
Batignolles-Monceau
Ménilmontant
Vaugirard
Reuilly
Hôtel-de-Ville
Observatoire
Gobelins
Luxembourg
Panthéon
Élysée
Passy
Opéra
Buttes-Montmartre
Temple
Palais-Bourbon


In [18]:
#print(transport_venues['Venue Category'].unique())
#print(transport_venues.shape)

#Data cleaning non-transports
transport_paris = transport_venues.loc[(transport_venues['Venue Category']=='Metro Station') | (transport_venues['Venue Category']=='Bus Stop') |(transport_venues['Venue Category']=='Tram Station') | (transport_venues['Venue Category']=='Train Station')]
transport_paris = transport_paris.rename(columns ={'Venue': 'station','Venue Latitude': 'lat_t', 'Venue Longitude': 'lon_t'})
#print(transport_paris.shape)
transport_paris.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,station,lat_t,lon_t,Venue Category
0,Louvre,48.862563,2.336443,"Métro Palais Royal – Musée du Louvre [1,7]",48.862604,2.336558,Metro Station
1,Louvre,48.862563,2.336443,Métro Louvre-Rivoli [1],48.860815,2.340991,Metro Station
2,Louvre,48.862563,2.336443,"Métro Pyramides [7,14]",48.865180,2.334654,Metro Station
3,Louvre,48.862563,2.336443,Métro Tuileries [1],48.864242,2.330439,Metro Station
4,Louvre,48.862563,2.336443,Métro Pont Neuf [7],48.858600,2.342255,Metro Station


With the data gathered so far, we can plot our first map. It shows: arrondissements/neighbourhoods classified by population density (2015), the geographic center of each arrondissement (blue dots), the green areas in metropolitan Paris (green dots) and the public transport (metro or bus) stations/stops.

In [21]:
paris_carte = folium.Map(location=[48.86, 2.34], zoom_start=12)

#Adding the arrondissements classified by population density 2015
folium.Choropleth(geo_data=coor_p_jsontxt,
     data=coor_p,
     columns=['geoid', 'Dens_2015'],
     key_on="feature.id",
     fill_opacity=0.5,
     line_opacity=0.2,
     line_color='white',
     line_weight=0,
     highlight=False,
     legend_name = 'Population density (hab/km2)',
     fill_color='YlOrRd').add_to(paris_carte)

# Adding the markers for coor_c (polygon centroids)
for lat, lng, label in zip(coor_c['lat'], coor_c['lon'], coor_c['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(paris_carte)

# Adding the markers for parks and gardens
for lat, lng, label in zip(garden_paris['lat_g'], garden_paris['lon_g'], garden_paris['garden']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#006600',
        fill_opacity=0.7,
        parse_html=False).add_to(paris_carte)
    
# Adding the markers for transports
for lat, lng, label in zip(transport_paris['lat_t'], transport_paris['lon_t'], transport_paris['station']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=3,
        popup=label,
        color='black',
        fill=True,
        fill_color='#808080',
        fill_opacity=0.7,
        parse_html=False).add_to(paris_carte)
    
paris_carte

# Results

From our first map, we can plot a second one which will be used to select the most suitable arrondissements to assess the best suitable locations. It is a heatmap of parks and gardens of Paris. It also includes the public transport stations network.

In [13]:
from folium import plugins
from folium.plugins import HeatMap

garden_XY = garden_paris[['lat_g','lon_g']].to_numpy() # Convert the coordinates into an array to speed up the computation time

garden_heatmap = folium.Map(location=[48.86, 2.34], zoom_start=12)
folium.TileLayer('cartodbpositron').add_to(garden_heatmap) # This line "blurs" the background of the map
folium.GeoJson(coor_p_jsontxt).add_to(garden_heatmap)

# Adding the markers for coor_c (polygon centroids)
for lat, lng, label in zip(coor_c['Lat'], coor_c['Lon'], coor_c['Nom']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(garden_heatmap)

# Adding the markers for transports
for lat, lng, label in zip(transport_paris['lat_t'], transport_paris['lon_t'], transport_paris['station']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=3,
        popup=label,
        color='black',
        fill=True,
        fill_color='#808080',
        fill_opacity=0.7,
        parse_html=False).add_to(garden_heatmap)

garden_heatmap.add_child(plugins.HeatMap(garden_XY, radius=20)) # Plots the gardens heatmap
    
garden_heatmap

To save some processing time, we create a dataframe with the arrondissements that we just selected. Popincourt is the most dense populated neighborhood in Paris, with Entrepôt and Temple within the top5. The maps shows that their green areas density is relatively low compared to other areas and because, due to their central location, they have a good public transport network. We add Bourse and Opéra to our selection because they lack of gardens. 

In [14]:
#Creation of two with the zones that are interesting to us

arr_garden_sel = garden_paris.loc[(garden_paris.Neighborhood== 'Entrepôt') |
                                  (garden_paris.Neighborhood== 'Temple') | (garden_paris.Neighborhood== 'Popincourt')|
                                  (garden_paris.Neighborhood== 'Bourse') | (garden_paris.Neighborhood== 'Opéra')]

arr_trans_sel = transport_paris.loc[(transport_paris.Neighborhood== 'Entrepôt') |
                                    (transport_paris.Neighborhood== 'Temple') | (transport_paris.Neighborhood== 'Popincourt') |
                                    (transport_paris.Neighborhood== 'Bourse') |(transport_paris.Neighborhood== 'Opéra')]

For the five selected neighbourhoods, we define a systematic point grid (a point every .002 degrees) that will allow us to determine the best locations for new green areas. We obtain a grid with 350 points.

In [15]:
#Create the point "grid" for the selected neighborhoods

cols = ['id','lat','lon']
grid_p = pd.DataFrame(columns = cols)

startx = 2.336 # We define the x's and y's limits to our point grid 
stopx = 2.385
step = 0.002 # We define the interval between points

float_range_arrayx = np.arange(startx, stopx, step) # Conversion into arrays to optimize the
float_range_listx = list(float_range_arrayx) # computation time
#print (float_range_listx[0])

starty= 48.857
stopy = 48.885

float_range_arrayy = np.arange(starty, stopy, step)
float_range_listy = list(float_range_arrayy)
#print (float_range_listy[0])

i=0
for x in float_range_listx:
    for y in float_range_listy: 
        grid_p.loc[i,'lat']=y
        grid_p.loc[i,'lon']=x
        grid_p.loc[i,'id'] = i
        i = i+1
#print (grid_p.head())

geo_grid = gpd.GeoDataFrame(grid_p, geometry=gpd.points_from_xy(grid_p.lon, grid_p.lat))
print(geo_grid) # Creation of a geodatafrme with the data we just got

geo_grid['id'] = geo_grid.index.astype(str)
geo_grid = geo_grid[['id','lat','lon','geometry']]
geo_grid_jsontxt = geo_grid.to_json() # We create the (geographic) information required to plot the data

      id     lat    lon                  geometry
0      0  48.857  2.336  POINT (2.33600 48.85700)
1      1  48.859  2.336  POINT (2.33600 48.85900)
2      2  48.861  2.336  POINT (2.33600 48.86100)
3      3  48.863  2.336  POINT (2.33600 48.86300)
4      4  48.865  2.336  POINT (2.33600 48.86500)
..   ...     ...    ...                       ...
345  345  48.875  2.384  POINT (2.38400 48.87500)
346  346  48.877  2.384  POINT (2.38400 48.87700)
347  347  48.879  2.384  POINT (2.38400 48.87900)
348  348  48.881  2.384  POINT (2.38400 48.88100)
349  349  48.883  2.384  POINT (2.38400 48.88300)

[350 rows x 4 columns]


In [16]:
grid_map = folium.Map(location=[48.87, 2.36], zoom_start=14)
folium.GeoJson(geo_grid_jsontxt).add_to(grid_map)
grid_map

Once we have our point grid, we can proceed to calculate the distance for each point to the closest garden and metro/bus stop.

In [17]:
# Calculation of minimum distances from the grid points to the closest garden/transport station

# add columns with radians for latitude and longitude
arr_garden_sel[['lat_rad_g','lon_rad_g']] = (np.radians(arr_garden_sel.loc[:,['lat_g','lon_g']]))

arr_trans_sel[['lat_rad_t','lon_rad_t']] = (np.radians(arr_trans_sel.loc[:,['lat_t','lon_t']]))

geo_grid['lat']=geo_grid['lat'].astype(float) # Convert object types into float
geo_grid['lon']=geo_grid['lon'].astype(float) 
geo_grid[['lat_rad','lon_rad']] = (np.radians(geo_grid.loc[:,['lat','lon']]))

dist = sklearn.neighbors.DistanceMetric.get_metric('haversine') # We find this workaround to calculate distances, since we are working with geographic data
dist_matrix_g = (dist.pairwise
    (arr_garden_sel[['lat_rad_g','lon_rad_g']],
     geo_grid[['lat_rad','lon_rad']])*6371) # Note that 3959 is the radius of the earth in km

df_dist_matrix_g = (pd.DataFrame(dist_matrix_g,index=arr_garden_sel['garden'], 
                 columns=geo_grid['id'])) #Dataframe that contains the distances

df_dist_long_g = (pd.melt(df_dist_matrix_g.reset_index(),id_vars='garden')) # DF containing all the distances to the grid points
df_dist_long_g = df_dist_long_g.rename(columns={'value':'dist_garden'})
mindist_garden=df_dist_long_g.loc[df_dist_long_g.groupby('id').dist_garden.idxmin()] # DF that contains the minimum distance to a garden for each grid point

C:\Users\Utilisateur\anaconda3\envs\geopandas_env\lib\site-packages\pandas\core\frame.py:3188: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Once we have the minimum distance to the closest garden for each grid point, we set up our first condition. Since we want to (eventually) build new green areas in those zones with a low density, we select those points that are, at least, 500m away from the closest garden or park. 500m means you'd have to walk 5 to 7 minutes, approximatively. There are 128 points which fulfill our first requirement.

In [18]:
#Select grid points which are farther than 500m of a garden
#So we can select the grid points which suit bests to our needs
# Areas without a garden in less than 500m but
# with transport stations closer than 200m

mindist_garden_sel = mindist_garden.loc[(mindist_garden.dist_garden > .500)]
mindist_garden_sel

,garden,id,dist_garden
8562,Jardin Nelson Mandela,103,0.626586
8645,Jardin Nelson Mandela,104,0.809508
8743,Square Saint-Laurent,105,0.734320
8826,Square Saint-Laurent,106,0.645788
8909,Square Saint-Laurent,107,0.627514
...,...,...,...
7664,Square Saint-Laurent,92,0.788165
7747,Square Saint-Laurent,93,0.773257
7830,Square Saint-Laurent,94,0.820710
7960,Square Louise Michel,95,0.773487


We do the same for the metro/bus stations or stops as for the gardens but with the condition that we want those points closer than 200m to a public transport stop. Everybody can walk 3 minutes to hop in the metro ride and ge to the closest park! There are 179 points which fulfill this second condition.

In [19]:
# We do the same but for transport
# We choose all grid points with a transport station 
#closer than 200m (2-3 min walk)
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix_t = (dist.pairwise
    (arr_trans_sel[['lat_rad_t','lon_rad_t']],
     geo_grid[['lat_rad','lon_rad']])*6371) # Note that 3959 is the radius of the earth in km

df_dist_matrix_t = (pd.DataFrame(dist_matrix_t,index=arr_trans_sel['station'], 
                 columns=geo_grid['id']))

df_dist_long_t = (pd.melt(df_dist_matrix_t.reset_index(),id_vars='station')) #Dataframe containing all the distances from the point grid
df_dist_long_t = df_dist_long_t.rename(columns={'value':'dist_station'})
mindist_trans=df_dist_long_t.loc[df_dist_long_t.groupby('id').dist_station.idxmin()] # Dataframe containing ONLY the min distances to each point

In [20]:
mindist_trans_sel = mindist_trans.loc[(mindist_trans.dist_station < .200)]
mindist_trans_sel

,station,id,dist_station
1684,Métro Saint-Georges [12],10,0.183413
16168,Métro Étienne Marcel [4],101,0.101537
16328,Métro Étienne Marcel [4],102,0.163160
16487,"Métro Réaumur—Sébastopol [3,4]",103,0.178277
16826,"Métro Bonne Nouvelle [8,9]",105,0.067735
...,...,...,...
14586,"Métro Bonne Nouvelle [8,9]",91,0.088702
15122,Métro Poissonnière [7],94,0.085431
15626,"Métro Barbès — Rochechouart [2,4]",97,0.146149
15692,"Métro Hôtel de Ville [1,11]",98,0.134667


We are looking for those points in the grid which fulfill the two conditions at the same time. That's why we have to merge/join the two DF we just created. This will allow us to identify the points that we are looking for. We end up with a total of 65 points that satisfy our two conditions at the same time. From these 65 points, there are three which are isolated. We decided to exclude them because they are to far aways from the others to be group with, when we apply the k-means method.

In [21]:
# Intersection (merge) of the two dataframes to select the
# grid points which are far from the gardens and parks but close
# to a metro/bus stop
grid_sel = pd.merge(mindist_garden_sel,mindist_trans_sel, on=['id'])
grid_sel

,garden,id,dist_garden,station,dist_station
0,Jardin Nelson Mandela,103,0.626586,"Métro Réaumur—Sébastopol [3,4]",0.178277
1,Square Saint-Laurent,105,0.734320,"Métro Bonne Nouvelle [8,9]",0.067735
2,Square Saint-Laurent,108,0.685142,Métro Poissonnière [7],0.076455
3,Square Saint-Laurent,109,0.802484,Métro Poissonnière [7],0.199749
4,Square Louise Michel,111,0.548537,"Métro Barbès — Rochechouart [2,4]",0.088982
...,...,...,...,...,...
60,Square Alex-Biscarre,80,0.701396,Métro Cadet [7],0.178312
61,Jardin Nelson Mandela,89,0.544034,Métro Sentier [3],0.135887
62,Jardin Nelson Mandela,90,0.747442,Métro Sentier [3],0.199816
63,Square Saint-Laurent,91,0.862204,"Métro Bonne Nouvelle [8,9]",0.088702


In [22]:
# Merging of the two dataframes, the one with the min distances to 
# gardens and the min distances to transport stations based on the
# id of the point grid
geo_grid_sel=pd.merge(grid_sel,geo_grid,on=['id'])
geo_grid_sel=geo_grid_sel.drop(['lat_rad','lon_rad'], axis = 1)
geo_grid_sel

geo_grid_sel = gpd.GeoDataFrame(geo_grid_sel, geometry='geometry')

geo_grid_sel_jsontxt = geo_grid_sel.to_json()

# We remove three isolated points
geo_grid_sel = geo_grid_sel.drop(geo_grid_sel.index[[19,29,35]], axis =0)

geo_grid_sel

,garden,id,dist_garden,station,dist_station,lat,lon,geometry
0,Jardin Nelson Mandela,103,0.626586,"Métro Réaumur—Sébastopol [3,4]",0.178277,48.867,2.350,POINT (2.35000 48.86700)
1,Square Saint-Laurent,105,0.734320,"Métro Bonne Nouvelle [8,9]",0.067735,48.871,2.350,POINT (2.35000 48.87100)
2,Square Saint-Laurent,108,0.685142,Métro Poissonnière [7],0.076455,48.877,2.350,POINT (2.35000 48.87700)
3,Square Saint-Laurent,109,0.802484,Métro Poissonnière [7],0.199749,48.879,2.350,POINT (2.35000 48.87900)
4,Square Louise Michel,111,0.548537,"Métro Barbès — Rochechouart [2,4]",0.088982,48.883,2.350,POINT (2.35000 48.88300)
...,...,...,...,...,...,...,...,...
60,Square Alex-Biscarre,80,0.701396,Métro Cadet [7],0.178312,48.877,2.346,POINT (2.34600 48.87700)
61,Jardin Nelson Mandela,89,0.544034,Métro Sentier [3],0.135887,48.867,2.348,POINT (2.34800 48.86700)
62,Jardin Nelson Mandela,90,0.747442,Métro Sentier [3],0.199816,48.869,2.348,POINT (2.34800 48.86900)
63,Square Saint-Laurent,91,0.862204,"Métro Bonne Nouvelle [8,9]",0.088702,48.871,2.348,POINT (2.34800 48.87100)


We plot our final results. First, a map with the 62 points and then, a second map with the clusters obtained from the k-means clustering method. We classified our 62 points into 10 clusters or groups.

In [23]:
# Creation of a map with the most appropriate zones for a garden
# from the point grid. These are the locations which are far away
# d>500m from a park but close to a transport station (d<200m), so
# they are accessible by foot
geo_grid_map = folium.Map(location=[48.87, 2.36], zoom_start=14)
folium.GeoJson(coor_p_jsontxt).add_to(geo_grid_map)

for lat, lng, label in zip(geo_grid_sel['lat'], geo_grid_sel['lon'], geo_grid_sel['id']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#006600',
        fill_opacity=0.7,
        parse_html=False).add_to(geo_grid_map)

geo_grid_map

In [24]:
# Creation of cluster of most suitable zones to locate a park
geo_grid_cluster = geo_grid_sel.drop(geo_grid_sel.index[[19,29,35]]) # Dropping isolated points

# set number of clusters
number_of_clusters = 10

good_xys = geo_grid_cluster[['lat', 'lon']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

cluster_map = folium.Map(location=[48.87, 2.36], zoom_start=14)
for lat, lon in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='red', fill=True, fill_opacity = .1).add_to(cluster_map) 
for lat, lon in zip(geo_grid_cluster['lat'],geo_grid_cluster['lon']):
    folium.CircleMarker([lat, lon], radius=3, color='green', fill=True, fill_color='#006600', fill_opacity=.7).add_to(cluster_map)

cluster_map

Finally, we obtain a set of 10 pairs of coordinates for the center of each cluster/group. Using reverse geocoding with Nominatim, we obtain an approximative address to each cluster center, represented by its geographic coordinates. These coordinates/addresses can be provided to the policy makers (city counselors) of Paris so they can decide where to start looking for the most suitable location for a new green zone in metropolitan Paris

In [25]:
# List of geographic coordinates of the cluster centers for the
# most suitable areas to locate parks

np_cluster_center = np.array(cluster_centers) 
np_cluster_center_round = np.around(np_cluster_center, 3) # Rounding decimal places to 3
df_cluster_center = pd.DataFrame(data=np_cluster_center_round, columns = ['lat','lon']) # Transforming the data into a dataframe

# Reverse geocoding with Nominatim 
# to obtain the address related to the geographic coordinates
geolocator = Nominatim(user_agent='rubengrd@hotmail.com')
df_cluster_center['address'] = df_cluster_center.apply(lambda row: geolocator.reverse((row['lat'], row['lon'])), axis=1)

df_cluster_center

,lat,lon,address
0,48.881,2.351,"(Les Demoiselles d'Honneur, Rue de Rocroy, Qua..."
1,48.866,2.381,"(Dèmonia, Avenue Jean Aicard, Quartier Saint-A..."
2,48.875,2.344,"(Grand Orient de France, 16, Rue Cadet, Quarti..."
3,48.870,2.345,"(Centre des Finances Publiques, Rue d'Uzès, Pa..."
4,48.882,2.367,"(231, Rue La Fayette, Quartier de l'Hôpital Sa..."
5,48.869,2.353,"(25, Rue Sainte-Apolline, Quartier des Arts-et..."
6,48.867,2.373,"(Relais Parmentier, Avenue Parmentier, Quartie..."
7,48.872,2.338,"(BNP Paribas, Rue Laffitte, Quartier du Faubou..."
8,48.881,2.358,"(no, Rue du Faubourg Saint-Denis, Quartier Sai..."
9,48.871,2.377,"(École maternelle Présentation, Rue de la Prés..."
